## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.08,-159.32,84.2,66,40,12.75,scattered clouds
1,1,Ushuaia,AR,-54.80,-68.30,50.0,93,90,9.17,light intensity drizzle
2,2,Sitka,US,57.05,-135.33,51.8,87,90,18.34,light rain
3,3,Bluff,NZ,-46.60,168.33,54.0,80,97,20.00,overcast clouds
4,4,Rocha,UY,-34.48,-54.33,53.6,87,3,4.76,clear sky


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.08,-159.32,84.20,66,40,12.75,scattered clouds
10,10,Gat,IL,31.61,34.76,75.00,100,4,1.99,clear sky
12,12,Rikitea,PF,-23.12,-134.97,75.67,67,0,5.61,clear sky
15,15,Tarakan,ID,3.30,117.63,78.42,83,93,8.12,overcast clouds
19,19,Hilo,US,19.73,-155.09,87.80,55,1,5.82,clear sky
20,20,Butaritari,KI,3.07,172.79,81.39,75,100,15.57,overcast clouds
21,21,Sao Raimundo Das Mangabeiras,BR,-7.02,-45.48,78.98,73,99,3.44,overcast clouds
27,27,Hambantota,LK,6.12,81.12,77.00,100,75,2.24,haze
28,28,Neiafu,TO,-18.65,-173.98,73.40,100,90,24.02,light rain
30,30,Ouahigouya,BF,13.58,-2.42,83.79,15,0,7.45,clear sky


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                273
City                   273
Country                270
Lat                    273
Lng                    273
Max Temp               273
Humidity               273
Cloudiness             273
Wind Speed             273
Current Description    273
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.08,-159.32,84.20,66,40,12.75,scattered clouds
10,10,Gat,IL,31.61,34.76,75.00,100,4,1.99,clear sky
12,12,Rikitea,PF,-23.12,-134.97,75.67,67,0,5.61,clear sky
15,15,Tarakan,ID,3.30,117.63,78.42,83,93,8.12,overcast clouds
19,19,Hilo,US,19.73,-155.09,87.80,55,1,5.82,clear sky
...,...,...,...,...,...,...,...,...,...,...
690,690,Pampa,US,35.54,-100.96,75.20,22,1,17.22,clear sky
693,693,Sattenapalle,IN,16.40,80.18,73.40,88,20,3.36,mist
702,702,San Andres,CO,12.58,-81.70,84.20,79,40,11.41,scattered clouds
703,703,Waingapu,ID,-9.66,120.26,79.07,75,100,7.45,overcast clouds


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.20,scattered clouds,22.08,-159.32,
10,Gat,IL,75.00,clear sky,31.61,34.76,
12,Rikitea,PF,75.67,clear sky,-23.12,-134.97,
15,Tarakan,ID,78.42,overcast clouds,3.30,117.63,
19,Hilo,US,87.80,clear sky,19.73,-155.09,
20,Butaritari,KI,81.39,overcast clouds,3.07,172.79,
21,Sao Raimundo Das Mangabeiras,BR,78.98,overcast clouds,-7.02,-45.48,
27,Hambantota,LK,77.00,haze,6.12,81.12,
28,Neiafu,TO,73.40,light rain,-18.65,-173.98,
30,Ouahigouya,BF,83.79,clear sky,13.58,-2.42,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.20,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
10,Gat,IL,75.00,clear sky,31.61,34.76,OlusHome
12,Rikitea,PF,75.67,clear sky,-23.12,-134.97,Pension Maro'i
15,Tarakan,ID,78.42,overcast clouds,3.30,117.63,RedDoorz Syariah near Taman Berlabuh Tarakan
19,Hilo,US,87.80,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
690,Pampa,US,75.20,clear sky,35.54,-100.96,Holiday Inn Express & Suites Pampa
693,Sattenapalle,IN,73.40,mist,16.40,80.18,Subhani hot hot tiffen ceanter
702,San Andres,CO,84.20,scattered clouds,12.58,-81.70,Hotel Arena Blanca
703,Waingapu,ID,79.07,overcast clouds,-9.66,120.26,Pondok Wisata Elim


In [24]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,84.20,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
10,Gat,IL,75.00,clear sky,31.61,34.76,OlusHome
12,Rikitea,PF,75.67,clear sky,-23.12,-134.97,Pension Maro'i
15,Tarakan,ID,78.42,overcast clouds,3.30,117.63,RedDoorz Syariah near Taman Berlabuh Tarakan
19,Hilo,US,87.80,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
20,Butaritari,KI,81.39,overcast clouds,3.07,172.79,Isles Sunset Lodge
21,Sao Raimundo Das Mangabeiras,BR,78.98,overcast clouds,-7.02,-45.48,Hotel Tropical
27,Hambantota,LK,77.00,haze,6.12,81.12,Bungalow 63
28,Neiafu,TO,73.40,light rain,-18.65,-173.98,Mystic Sands
30,Ouahigouya,BF,83.79,clear sky,13.58,-2.42,Hotel Liberte


In [25]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))